# Gradient descent solutions to most likely hidden state by MLE/MAP estimation

This notebook shows how one can use maximum likelihood estimation (MLE) or maximum a posteriori (MAP) estimation to find the most likely hidden state given some sensory observation via gradient descent instead of using the analytic solution.

==========================================================================

* **Notebook dependencies**:
    * ...

* **Content**: Jupyter notebook accompanying Chapter 2 of the textbook "Fundamentals of Active Inference"

* **Author**: Sanjeev Namjoshi (sanjeev.namjoshi@gmail.com)

* **Version**: 0.1